

```
# 코드로 형식 지정됨
```

# KoBERT finetuning

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zfqmbem8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zfqmbem8
     |████████████████████████████████| 132 kB 3.9 MB/s 
     |████████████████████████████████| 344 kB 34.6 MB/s 
     |████████████████████████████████| 47.3 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 26.5 MB/s 
     |████████████████████████████████| 1.2 MB 36.8 MB/s 
     |████████████████████████████████| 3.8 MB 36.0 MB/s 
     |████████████████████████████████| 6.5 MB 41.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 8.6 MB 43.0 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 127 kB 48.5 MB/s 
  Created wheel for kobert: filename=kob

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1Cj9DEvqtsjQd6d_HN1D2l69WJaHDqfUG" -O ratings_train.txt
!wget "https://drive.google.com/uc?export=download&id=160lKBgDJ6kOGolnt1-q3mPKYFVYVzdvs" -O ratings_test.txt

--2022-03-19 14:47:26--  https://drive.google.com/uc?export=download&id=1Cj9DEvqtsjQd6d_HN1D2l69WJaHDqfUG
Resolving drive.google.com (drive.google.com)... 66.102.1.102, 66.102.1.101, 66.102.1.139, ...
Connecting to drive.google.com (drive.google.com)|66.102.1.102|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-03-19 14:47:26 ERROR 404: Not Found.

--2022-03-19 14:47:26--  https://drive.google.com/uc?export=download&id=160lKBgDJ6kOGolnt1-q3mPKYFVYVzdvs
Resolving drive.google.com (drive.google.com)... 66.102.1.139, 66.102.1.102, 66.102.1.113, ...
Connecting to drive.google.com (drive.google.com)|66.102.1.139|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-03-19 14:47:26 ERROR 404: Not Found.



In [ ]:
dataset_train = nlp.data.TSVDataset("train_sample.txt", field_indices=[1,2], num_discard_samples=1,encoding = 'euc-kr')
dataset_test = nlp.data.TSVDataset("test_sample.txt", field_indices=[1,2], num_discard_samples=1,encoding = 'euc-kr')

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.716132640838623 train acc 0.5
epoch 1 batch id 201 loss 0.2588975727558136 train acc 0.6497201492537313
epoch 1 batch id 401 loss 0.3790135979652405 train acc 0.7350374064837906
epoch 1 batch id 601 loss 0.39951932430267334 train acc 0.769082778702163
epoch 1 train acc 0.77205


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 1 test acc 0.8653463375796179


...

epoch 4 test acc 0.8810708598726115


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.005820595193654299 train acc 1.0
epoch 5 batch id 201 loss 0.012925940565764904 train acc 0.9713930348258707
epoch 5 batch id 401 loss 0.03088664822280407 train acc 0.9760754364089775
epoch 5 batch id 601 loss 0.01171417161822319 train acc 0.9776674292845258
epoch 5 train acc 0.9779


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 test acc 0.8781847133757962


In [ ]:
import pandas as pd
df = pd.read_csv('df_notk.csv', encoding = 'cp949')

In [ ]:
import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  for c in comment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

0       판타지를 넘어 영화 역사에 남을 명작이다. 내가 이걸 왜 극장에서 못봤을까.... ...
1                                           이집트여행하는느낌의 영화
2                                  목포 연설 장면은 넋을 잃고 보게 된다.
3                                     그저 그렇네요. 뻔한 반전과 결말.
4                     더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천
                              ...                        
9325                                이해할 수 없는 남자와 이해해주는 여자
9326                   배우들 연기력이 장난아님. 역시 최민식 배우는 클라스가 다름!
9327    그래도 최근에 나왔던 영화들에 비해 내용이 되게 흥미진진했음 리암 니슨이 덜뛰는 대...
9328                        누구나 시작은 어설픈 법. 그래서 더 매력적인 히어로
9329                                  병과 싸우기 위해서는 공부해야 한다
Name: review, Length: 9330, dtype: object

In [ ]:

df_list = []
df_review = df['review']

for i in range(len(df)) :
  df_list = df_review.values.tolist()




  

In [ ]:
df_list

['판타지를 넘어 영화 역사에 남을 명작이다. 내가 이걸 왜 극장에서 못봤을까.... 폰으로 봤을때의 감동과는 비교가 안될텐데....',
 '이집트여행하는느낌의 영화',
 '목포 연설 장면은 넋을 잃고 보게 된다.',
 '그저 그렇네요. 뻔한 반전과 결말.',
 '더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천',
 '전 올레tv로 몇년전에 재밌게 봤다가 방금 소장용으로 재구입하여 또 봤어요. 전 재밌게 봤는데요..부인인 조여정은 송승헌의 섹슈얼한 이상형은 아니었던 것 같아요. 극중 조여정과 임지연은 성격이나 분위기가 확 다르죠. 그리고 봐보면 극중 임지연은 겉으론 조용하고 청순해보여도 은밀하게는 과감하게 그러나 조용히 끼를 부립니다. 속된 말로, 호박씨까는스타일이죠ㅋ 그런 스타일에 꽂혔을 수 있고 이상형인듯 거의 뭐 첫눈에 반한 거에 준해서 빠져들어간 것 같네요. 송승헌이 임지연한테. 저는 꼭 이 이상의 대단한 사랑의 동기?가 있어야만 납득가능하다고 보진 않네요. 왜 저렇게까지 극중 임지연을 사랑하는가가 포인트인 게 아니라, 그냥 송승헌은 이미 임지연이 좋은거고, 많이 좋아하다보니 저렇게까지 다 걸어버리는 송승헌 자체의 캐릭터를 저는 감상한거죠. 그리고 이런 러브스토리를 기반으로 했기에 정사씬이 참 야하군여. 크하하',
 '평범한 나라, 평범한 가정, 평범한 아이로 태어남에 감사합니다.',
 '반전을 거듭하는건 좋은데 마지막반전은 좀 sf영화도 아니고 에바였지 않나...',
 'ㅠㅠ 영화관에서 잠든 유일한 영화..',
 '생각보다 흡입력 있고 지루하지 않고 볼만하다',
 '아수라2냐? 이것도 이재명 주연이다음주운전에 주변사람 살인으로 위장하고 뭐 대단하다 이배우',
 '이런 비극이 없도록 전쟁은 일어나면 안되는데.. 현재 고통받는 우크라이나가 생각나에요..',
 '주연들은 매력적이다, 이런 영화 특징답게 스토리나 개연성은 크게 보지 마시고 단순히 재미로 킬링타임하고 싶으시다면 보시는 거 추천',
 '조인성 연기 커리어의 정점이다.. 다리도 길

In [ ]:
#dict(df_list,getSentimentValue(df_list, tok, max_len, batch_size, device))
getSentimentValue(df_list, tok, max_len, batch_size, device)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
